In [7]:
import vk
from time import sleep
import pandas as pd

In [2]:
#сюда вводить токен в кавычках
token = ''

In [3]:
session = vk.Session(access_token = token) 
api = vk.API(session)

In [30]:
# сюда id группы в кавычках
group = ''
# сюда численный id группы со знаком - перед цифрами, узнать можно тут https://regvk.com/id/
group1 = -

In [17]:
# first 100
res = api.wall.get(domain = group, count = 100, v = 5.126)
items = res['items']

ids = []

for item in items:
    L = [item['id'], item['date']]
    ids.append(L) 

In [18]:
for i in range(1, 7):
    res2 = api.wall.get(domain = group, count = 100, offset = 100 * i, v = 5.126)
    items2 = res2['items']
    items.extend(items2)
    sleep(0.5)

In [19]:
for item in items:
    L = [item['id'], item['date']]
    ids.append(L)

In [47]:
df = pd.DataFrame(ids)
df.columns = ['ids', 'time']

In [98]:
posts = df['ids']

In [99]:
posts = pd.DataFrame(posts)

In [100]:
def split_dataframe(df, chunk_size = 30): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [107]:
dfs = split_dataframe(posts)

In [114]:
lst = []
for i in dfs:
    lst.append(list(i['ids']))

In [129]:
exm = lst[0]
st = api.stats.getPostReach(owner_id = group1, post_ids = exm, v = 5.126)
new_dict = {item['post_id']:item for item in st}

In [134]:
data = []
for i in lst:
    stats = api.stats.getPostReach(owner_id = group1, post_ids = i, v = 5.126)
    new_dict = {item['post_id']:item for item in stats}
    res = new_dict.values()
    data.extend(res)

In [135]:
reach = []
for item in data:
    l = [item['reach_subscribers'], item['reach_total'], item['reach_ads'],
        item['reach_viral'], item['links']]
    reach.append(l)

In [138]:
columns = ['reach_subscribers', 'reach_total', 'reach_ads', 'reach_viral', 'links']
ndata = pd.DataFrame(reach, columns = columns)

In [148]:
dfs = [df,ndata]
ndata1 = pd.concat(dfs, axis=1)

In [153]:
# timestamp 01.01.2020 1577865600
df = ndata1.drop(ndata1['time'] < 1577865600, errors='ignore') #удалить лишние посты (до 1 января 2020 года)

In [155]:
df.to_csv('vkdata.csv')